<a href="https://colab.research.google.com/github/atlas-github/nus_cloudcomputing/blob/main/Google_News_RSS_Feed_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser pymssql

In [ ]:
import feedparser
import pandas as pd
import pymssql

# RSS feed URL
rss_url = "https://news.google.com/rss/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx1YlY4U0FtVnVHZ0pWVXlnQVAB?hl=en-US&gl=US&ceid=US:en"

# Azure SQL Database setup
server = '### INSERT SERVER NAME ###.database.windows.net'
database = '### INSERT DATABASE NAME ###'
username = '### INSERT SQL SERVER USERNAME ###'  # Optional, if required by your Azure SQL
password = '### INSERT SQL SERVER PASSWORD ###'  # Optional, if required by your Azure SQL

# Create a connection to Azure SQL Database
conn = pymssql.connect(server = server,
                       user = username,
                       password = password,
                       database = database)

cursor = conn.cursor()

# Email setup
smtp_server = "smtp.gmail.com"
smtp_port = 587
email_sender = "### INSERT SENDER EMAIL ###"
email_password = "### INSERT APP PASSWORD FROM GOOGLE ACCOUNT ###"
email_recipient = "### INSERT RECIPIENT EMAIL ###"

# Parse the RSS feed
feed = feedparser.parse(rss_url)
entries = []
for entry in feed.entries:
    entries.append({
        "Title": entry.title,
        "Link": entry.link,
        "Published": entry.published,
        "Summary": entry.summary if "summary" in entry else ""
    })

# Create a DataFrame
df = pd.DataFrame(entries)

# Query to check for existing entries in Azure SQL
existing_query = """
    SELECT Link FROM rss_feed_entries
"""
cursor.execute(existing_query)
existing_links = [row[0] for row in cursor.fetchall()]

# Find new rows (not in existing data)
new_rows = df[~df["Link"].isin(existing_links)]

# Insert new rows into Azure SQL Database
if not new_rows.empty:
    for index, row in new_rows.iterrows():
        insert_query = """
            INSERT INTO rss_feed_entries (Title, Link, Published, Summary)
            VALUES (%s, %s, %s, %s)
        """
        cursor.execute(insert_query, (row["Title"], row["Link"], row["Published"], row["Summary"]))
    conn.commit()  # Commit the transaction

    # Send email notification for new rows
    html_table = new_rows.to_html(index=False, escape=False)
    email_subject = "New RSS Feed Entries"
    email_body = f"""
    New entries have been added to the RSS feed:
    {html_table}
    """

    # Create the email message
    message = MIMEMultipart()
    message["From"] = email_sender
    message["To"] = email_recipient
    message["Subject"] = email_subject
    message.attach(MIMEText(email_body, "html"))

    # Send the email
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(email_sender, email_password)
        server.sendmail(email_sender, email_recipient, message.as_string())

    print("New rows inserted into Azure SQL Database and email notification sent.")
else:
    print("No new rows to insert.")


New rows inserted into Azure SQL Database and email notification sent.
